In [3]:
%pip install ipykernel
%pip install tensorflow
%pip install numpy
%pip install keras-tuner
%pip install scikit-learn
%pip install pandas
%pip install --upgrade ml_dtypes

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for ml_dtypes from https://files.pythonhosted.org/packages/81/1c/d27a930bca31fb07d975a2d7eaf3404f9388114463b9f15032813c98f893/ml_dtypes-0.5.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/206.3 kB ? eta -:--:--
   ----------------------------------- ---- 184.3/206.3 kB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 206.3/206.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Acesso negado: 'C:\\Users\\adria\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\~l_dtypes\\_ml_dtypes_ext.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import Hyperband
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow.keras.backend as K
from keras_tuner import Objective



In [2]:
df = pd.read_csv('df_tratado.csv', sep=',')
df.head()

,Unnamed: 0,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,...,latitude,longitude,regional,delegacia,uop,origem_arquivo,dia,mes,ano,grave
0,1,405158.0,2022-01-01,Sábado,02:40:00,PR,116,33.0,CAMPINA GRANDE DO SUL,Ingestão de álcool pelo condutor,...,"-25,114403","-48,8467554",SPRF-PR,DEL01-PR,UOP02-DEL01-PR,datatran2022.csv,1,1,2022,1
1,28,405442.0,2022-02-01,Domingo,12:00:00,SC,116,54.0,PAPANDUVA,Velocidade Incompatível,...,"-26,39448417","-50,16480003",SPRF-SC,DEL06-SC,UOP02-DEL06-SC,datatran2022.csv,1,2,2022,0
2,34,405542.0,2022-02-01,Domingo,18:40:00,RS,290,112.0,ELDORADO DO SUL,Acessar a via sem observar a presença dos outr...,...,"-30,040455","-51,329193",SPRF-RS,DEL02-RS,UOP01-DEL02-RS,datatran2022.csv,1,2,2022,0
3,36,405558.0,2022-02-01,Domingo,23:00:00,SC,101,118.0,ITAJAI,Ingestão de álcool pelo condutor,...,"-26,895859","-48,71903",SPRF-SC,DEL04-SC,UOP04-DEL04-SC,datatran2022.csv,1,2,2022,0
4,38,405589.0,2022-03-01,Segunda,03:30:00,SC,101,329.0,CAPIVARI DE BAIXO,Ingestão de álcool pelo condutor,...,"-28,45039264","-48,97180974",SPRF-SC,DEL02-SC,UOP01-DEL02-SC,datatran2022.csv,1,3,2022,0


In [14]:
features = ['dia_semana','fase_dia','condicao_metereologica','tipo_pista','veiculos']
X = df[features]
y = df['grave']

In [4]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [5]:
X_encoded.head()

,veiculos,dia_semana_Quarta,dia_semana_Quinta,dia_semana_Segunda,dia_semana_Sexta,dia_semana_Sábado,dia_semana_Terça,fase_dia_Anoitecer,fase_dia_Plena Noite,fase_dia_Pleno dia,condicao_metereologica_Neblina,condicao_metereologica_Sol,tipo_pista_Múltipla,tipo_pista_Simples
0,2,False,False,False,False,True,False,False,False,True,False,True,False,False
1,2,False,False,False,False,False,False,False,False,True,False,True,False,True
2,5,False,False,False,False,False,False,True,False,False,False,True,False,True
3,2,False,False,False,False,False,False,False,True,False,False,True,False,False
4,3,False,False,True,False,False,False,False,True,False,False,True,False,False


In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for i in range(hp.Int('num_layers', 1, 7)):
        model.add(
            layers.Dense(
                units=hp.Int(f'units_{i}', min_value=16, max_value=128, step=16),
                activation=hp.Choice('activation', ['relu', 'tanh'])
            )
        )
    
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=keras.optimizers.Adam(hp.Float('lr', 1e-4, 1e-2, sampling='log')),
        loss='binary_crossentropy',
        metrics=[f1_score]
    )
    return model

In [9]:
def f1_score(y_true, y_pred):
    y_pred_bin = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred_bin, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_bin, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_bin), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

In [10]:
tuner = Hyperband(
    build_model,
    objective=Objective("val_f1_score", direction="max"),  # <- agora funciona!
    max_epochs=30,
    factor=3,
    directory='hyperband_dir',
    project_name='modelo_acidente_f1'
)

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [12]:
tuner.search(X_train, y_train, epochs=30, validation_split=0.2, callbacks=[stop_early])

Trial 2 Complete [00h 00m 01s]

Best val_f1_score So Far: None
Total elapsed time: 00h 00m 04s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
1                 |3                 |num_layers
80                |32                |units_0
relu              |tanh              |activation
0.00012317        |0.00010552        |lr
112               |16                |units_1
112               |16                |units_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


Traceback (most recent call last):
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\adria\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_

In [69]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
print("📊 Melhores hiperparâmetros encontrados:")
for param, value in best_hp.values.items():
    print(f"{param}: {value}")


📊 Melhores hiperparâmetros encontrados:
num_layers: 1
units_0: 112
activation: relu
lr: 0.002579876453443278
units_1: 64
units_2: 96
units_3: 112
units_4: 32
units_5: 80
units_6: 96
tuner/epochs: 4
tuner/initial_epoch: 0
tuner/bracket: 2
tuner/round: 0


In [70]:

history = best_model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test))

# ==== 6. Avaliar ====
loss, f1 = best_model.evaluate(X_test, y_test)
print(f'\n✅ F1-score no teste: {f1:.4f}')

Epoch 1/150
45005/45005 [==============================] - 39s 859us/step - loss: 0.1630 - f1_score: 2.5849e-04 - val_loss: 0.1631 - val_f1_score: 3.4068e-04
Epoch 2/150
45005/45005 [==============================] - 38s 851us/step - loss: 0.1629 - f1_score: 1.8887e-04 - val_loss: 0.1630 - val_f1_score: 8.8873e-05
Epoch 3/150
45005/45005 [==============================] - 39s 860us/step - loss: 0.1630 - f1_score: 1.9257e-04 - val_loss: 0.1632 - val_f1_score: 8.8873e-05
Epoch 4/150
45005/45005 [==============================] - 39s 861us/step - loss: 0.1629 - f1_score: 1.4147e-04 - val_loss: 0.1632 - val_f1_score: 0.0000e+00
Epoch 5/150
45005/45005 [==============================] - 38s 853us/step - loss: 0.1628 - f1_score: 1.7776e-04 - val_loss: 0.1629 - val_f1_score: 2.3699e-04
Epoch 6/150
45005/45005 [==============================] - 39s 857us/step - loss: 0.1628 - f1_score: 1.8887e-04 - val_loss: 0.1629 - val_f1_score: 1.4812e-04
Epoch 7/150
45005/45005 [===========================